In [1]:
pwd

'/net/home/dmederer/Transferability_efficient'

In [2]:
from data_module_F import *
from feature_module_F import *
#from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneGroupOut
import numpy as np
import pandas as pd

import os

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [3]:
#path_file = '/net/home/dmederer/Efficient_net_new_model_folder/Eya_data_new.csv' ##insert ### path to raw data
path_n =  '/net/home/dmederer/Transferability_efficient/' ##insert ### directory of the final prepared data


if not(os.path.exists(path_n)):
    os.mkdir(path_n) 

In [4]:
# Eya´s data

file = 'test_new_corrected.csv'
path_file = '/net/home/dmederer/Transferability_efficient/{}'.format(file)
db1, features1, labels1 = read_db(path_file, sp=True)

/net/home/dmederer/Transferability_efficient/data_module_F.py:62: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  db = pd.read_csv(file, encoding=encoding)


In [5]:
db1

,Species,dataset,numSamples,LandCover,Tool,Site,Year,Al content (mg/cm²),Aluminum concentration (mg/g),Anthocyanin content (μg/cm²),...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,Agrostis sp,1,73,Grassland,Proximal,Belgium,2016.0,NaN,NaN,NaN,...,-0.622095,-0.635702,-0.510096,-0.427374,-0.381772,-0.360621,-0.336525,-0.253797,-0.104727,-0.085622
1,Agrostis sp,1,73,Grassland,Proximal,Belgium,2016.0,NaN,NaN,NaN,...,0.029224,-0.068101,-0.144032,-0.216670,-0.176655,-0.112455,-0.048547,0.007510,0.037106,0.036519
2,Agrostis sp,1,73,Grassland,Proximal,Belgium,2016.0,NaN,NaN,NaN,...,-0.071035,-0.160926,-0.103561,-0.035172,0.050518,0.140078,0.192099,0.215595,0.211715,0.161934
3,Agrostis sp,1,73,Grassland,Proximal,Belgium,2016.0,NaN,NaN,NaN,...,-0.668511,0.168098,0.270497,0.052172,-0.430093,-0.321831,-0.231444,-0.233715,-0.055118,-0.006315
4,Angelica sylvestris,1,73,Grassland,Proximal,Belgium,2016.0,NaN,NaN,NaN,...,0.035840,0.047321,0.077928,0.106978,0.083175,0.051137,0.042802,0.047886,0.045793,0.033436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5568,NaN,42,63,Mix,Airborne,D19,NaN,NaN,NaN,NaN,...,0.028365,0.028024,0.031615,0.035206,0.038797,0.042388,0.045979,0.042697,0.039414,0.036132
5569,NaN,42,63,Mix,Airborne,D19,NaN,NaN,NaN,NaN,...,0.027766,0.027159,0.029893,0.032627,0.035361,0.038095,0.040829,0.040883,0.040937,0.040992
5570,NaN,42,63,Forest,Airborne,D19,NaN,0.002934,0.09154,NaN,...,0.036352,0.039523,0.038079,0.036634,0.035190,0.033746,0.032301,0.032271,0.032241,0.032210
5571,NaN,42,63,Mix,Airborne,D19,NaN,NaN,NaN,NaN,...,0.037825,0.039573,0.042541,0.045510,0.048478,0.051446,0.054414,0.052682,0.050949,0.049217


In [7]:
# In db1, show all columns that start with L
db1.filter(regex='^L', axis=1)



,LandCover,LAI (m²/m²),LMA (g/m²),LDMC (g/g),LWC (%),Lignin (mg/cm²),Lignin (mg/g)
0,Grassland,NaN,19.004734,0.239945,76.005502,NaN,NaN
1,Grassland,NaN,27.890214,0.296497,70.350301,NaN,NaN
2,Grassland,NaN,35.968971,0.352305,64.769529,NaN,NaN
3,Grassland,NaN,31.493903,0.307420,69.257998,NaN,NaN
4,Grassland,NaN,50.769950,0.285387,71.461275,NaN,NaN
...,...,...,...,...,...,...,...
5568,Mix,NaN,85.446279,NaN,NaN,1.642638,192.242179
5569,Mix,NaN,58.524372,NaN,NaN,1.148454,196.235148
5570,Forest,NaN,320.524111,NaN,NaN,9.292787,289.924734
5571,Mix,NaN,85.529811,NaN,NaN,1.904810,222.707135


In [10]:
# In db1, find out the number of entries in Anthocyanin, LAI, and LMA
# calculate the total number of entries in the column
db1['Anthocyanin content (μg/cm²)'].value_counts().sum()

644

In [11]:
db1['LAI (m²/m²)'].value_counts().sum()


1640

In [12]:
db1['LMA (g/m²)'].value_counts().sum()

3328

In [6]:
# Hybrid data preparation real data # 

w1 = db1.loc[:,:'Site']
#w = db.loc[:,:'plot'] # Alternative


inter = feature_preparation(features1, order=1)

# list_clean = [1772,1774,2001,3953,3952,3956,5331]+ [i for i in range(1653,1677)]+[i for i in range(1775,1778)] ######## this is specific to the last version of data
# cleansp = inter.drop(list_clean, axis = 0).copy() ######## this is specific to the last version of data

cleansp1 = inter.copy()
cleanlb1 = labels1.loc[cleansp1.index,:].reset_index(drop=True)
cleanw1 = w1.loc[cleansp1.index,:].reset_index(drop=True)

X1 = pd.concat([cleanlb1, cleansp1.reset_index(drop=True)], axis=1, join='inner')
# X.to_csv(path_n + 'completeClean.csv')

X1 = X1.drop('Year', axis=1) # extra column might cause training issues

# Save processed original data
X1.to_csv("processed_original_data.csv")

X1

,Species,dataset,numSamples,LandCover,Tool,Site,Al content (mg/cm²),Aluminum concentration (mg/g),Anthocyanin content (μg/cm²),Anthocyanin concentration (mg/g),...,2441,2442,2443,2444,2445,2446,2447,2448,2449,2450
0,Agrostis sp,1,73,Grassland,Proximal,Belgium,NaN,NaN,NaN,NaN,...,0.036088,0.035571,0.035054,0.034538,0.034021,0.033504,0.032987,0.032470,0.031953,0.031436
1,Agrostis sp,1,73,Grassland,Proximal,Belgium,NaN,NaN,NaN,NaN,...,0.039821,0.039113,0.038406,0.037699,0.036991,0.036284,0.035576,0.034869,0.034161,0.033454
2,Agrostis sp,1,73,Grassland,Proximal,Belgium,NaN,NaN,NaN,NaN,...,0.090733,0.090244,0.089754,0.089264,0.088775,0.088285,0.087795,0.087306,0.086816,0.086327
3,Agrostis sp,1,73,Grassland,Proximal,Belgium,NaN,NaN,NaN,NaN,...,0.032456,0.032020,0.031585,0.031149,0.030714,0.030278,0.029843,0.029408,0.028972,0.028537
4,Angelica sylvestris,1,73,Grassland,Proximal,Belgium,NaN,NaN,NaN,NaN,...,0.032566,0.032239,0.031912,0.031585,0.031258,0.030931,0.030604,0.030277,0.029950,0.029623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5568,NaN,42,63,Mix,Airborne,D19,NaN,NaN,NaN,NaN,...,0.034735,0.034522,0.034310,0.034098,0.033886,0.033674,0.033462,0.033250,0.033038,0.032826
5569,NaN,42,63,Mix,Airborne,D19,NaN,NaN,NaN,NaN,...,0.038171,0.038016,0.037861,0.037705,0.037550,0.037395,0.037239,0.037084,0.036929,0.036774
5570,NaN,42,63,Forest,Airborne,D19,0.002934,0.09154,NaN,NaN,...,0.031313,0.031272,0.031232,0.031191,0.031150,0.031110,0.031069,0.031029,0.030988,0.030947
5571,NaN,42,63,Mix,Airborne,D19,NaN,NaN,NaN,NaN,...,0.044352,0.044252,0.044152,0.044053,0.043953,0.043854,0.043754,0.043655,0.043555,0.043455


In [13]:
# Read in RTM data #

#db = pd.read_csv("splot_rtm_data_5d.csv")
db, features, labels = read_db("splot_rtm_data_grass_forest_shrub_balanced.csv", sp=True)

In [14]:
db

,PlotObservationID,Chl content (μg/cm²),N content (mg/cm²),LMA (g/m²),EWT (mg/cm²),Anthocyanin content (μg/cm²),Carotenoid content (μg/cm²),LAI (m²/m²),400,401,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,1512,41.25216,0.162713,61.24437,9.76991,1.37828,9.83397,2.43228,0.018934,0.018928,...,0.037029,0.036804,0.036674,0.036470,0.036404,0.036336,0.036159,0.036074,0.035927,0.035855
1,1514,41.44864,0.118076,64.26408,10.58948,1.34498,9.71677,5.52188,0.019123,0.019114,...,0.033504,0.033287,0.033163,0.032966,0.032901,0.032836,0.032665,0.032583,0.032441,0.032370
2,1517,44.88884,0.113763,71.05461,14.13997,1.12516,8.69746,5.43700,0.018604,0.018595,...,0.025959,0.025796,0.025703,0.025557,0.025508,0.025456,0.025327,0.025262,0.025154,0.025098
3,1628,37.42986,0.126968,62.64776,11.36493,1.38656,9.95212,2.60911,0.017203,0.017197,...,0.031878,0.031700,0.031598,0.031437,0.031385,0.031327,0.031183,0.031111,0.030991,0.030929
4,1635,39.98550,0.162943,56.75211,17.10991,1.62973,10.88106,6.34486,0.016754,0.016748,...,0.021612,0.021498,0.021433,0.021331,0.021297,0.021258,0.021164,0.021116,0.021036,0.020994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20195,1126015,36.03518,0.149128,62.41111,19.54961,1.24821,9.09141,3.55873,0.102893,0.102705,...,0.194174,0.194163,0.194160,0.194149,0.194150,0.193940,0.193766,0.193556,0.193343,0.193175
20196,1126086,35.08011,0.149069,70.76630,19.30091,1.15780,8.61656,4.66223,0.063157,0.063071,...,0.128498,0.128474,0.128467,0.128447,0.128453,0.128324,0.128207,0.128080,0.127945,0.127851
20197,1126605,33.52243,0.142339,64.56431,19.19238,1.08414,8.19659,2.84366,0.059528,0.059438,...,0.113018,0.112985,0.112970,0.112942,0.112939,0.112819,0.112704,0.112583,0.112453,0.112359
20198,1126709,37.77446,0.164535,84.06026,20.10563,1.17632,8.63782,3.66339,0.077145,0.077020,...,0.149373,0.149353,0.149347,0.149330,0.149333,0.149176,0.149040,0.148884,0.148722,0.148601


In [15]:
# Preprocessing RTM data # 


# rename 'plot' column to 'Year' in RTM data => enables inner join with real data later # deprecated
db = db.rename(columns={'PlotObservationID': 'Year'})

# concatenate dataframes along rows and fill missing values with NaN
db_combined = pd.concat([db1, db], axis=0, sort=False)

# fill NaNs in Site column to prevent errors

db_combined.Site = db_combined.Site.fillna("empty")

# give all RTM data a dataset number

db_combined.dataset = db_combined.dataset.fillna(43) ### depends on how many datasets are in the original data!

# give all RTM data a dataset number

db_combined.numSamples = db_combined.numSamples.fillna(16482) ### DEPENDS on the number of rows in the added RTM data!

#db_combined = db_combined.drop('PlotObservationID', axis=1) # if there, extra column causes error in preprocessing

# Save and Reload for the custom functions to work!
db_combined.to_csv(path_n + 'db_combined.csv')

db_combined, features_combined, labels_combined = read_db('db_combined.csv', sp=True)



/net/home/dmederer/Transferability_efficient/data_module_F.py:62: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  db = pd.read_csv(file, encoding=encoding)


In [16]:
# Remove all real data entries from the data frame, but keep the necessary column structure
db = db_combined.loc[5573:,:].reset_index(drop = True) # remove the index to prevent errors
features = features_combined.loc[5573:,:].reset_index(drop = True)
labels = labels_combined.loc[5573:,:].reset_index(drop = True)
db

,Species,dataset,numSamples,LandCover,Tool,Site,Year,Al content (mg/cm²),Aluminum concentration (mg/g),Anthocyanin content (μg/cm²),...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,NaN,43.0,16482.0,NaN,NaN,empty,1512.0,NaN,NaN,1.37828,...,0.037029,0.036804,0.036674,0.036470,0.036404,0.036336,0.036159,0.036074,0.035927,0.035855
1,NaN,43.0,16482.0,NaN,NaN,empty,1514.0,NaN,NaN,1.34498,...,0.033504,0.033287,0.033163,0.032966,0.032901,0.032836,0.032665,0.032583,0.032441,0.032370
2,NaN,43.0,16482.0,NaN,NaN,empty,1517.0,NaN,NaN,1.12516,...,0.025959,0.025796,0.025703,0.025557,0.025508,0.025456,0.025327,0.025262,0.025154,0.025098
3,NaN,43.0,16482.0,NaN,NaN,empty,1628.0,NaN,NaN,1.38656,...,0.031878,0.031700,0.031598,0.031437,0.031385,0.031327,0.031183,0.031111,0.030991,0.030929
4,NaN,43.0,16482.0,NaN,NaN,empty,1635.0,NaN,NaN,1.62973,...,0.021612,0.021498,0.021433,0.021331,0.021297,0.021258,0.021164,0.021116,0.021036,0.020994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20195,NaN,43.0,16482.0,NaN,NaN,empty,1126015.0,NaN,NaN,1.24821,...,0.194174,0.194163,0.194160,0.194149,0.194150,0.193940,0.193766,0.193556,0.193343,0.193175
20196,NaN,43.0,16482.0,NaN,NaN,empty,1126086.0,NaN,NaN,1.15780,...,0.128498,0.128474,0.128467,0.128447,0.128453,0.128324,0.128207,0.128080,0.127945,0.127851
20197,NaN,43.0,16482.0,NaN,NaN,empty,1126605.0,NaN,NaN,1.08414,...,0.113018,0.112985,0.112970,0.112942,0.112939,0.112819,0.112704,0.112583,0.112453,0.112359
20198,NaN,43.0,16482.0,NaN,NaN,empty,1126709.0,NaN,NaN,1.17632,...,0.149373,0.149353,0.149347,0.149330,0.149333,0.149176,0.149040,0.148884,0.148722,0.148601


In [17]:
features

,400,401,402,403,404,405,406,407,408,409,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,0.018934,0.018928,0.018922,0.018919,0.018917,0.018919,0.018889,0.018859,0.018834,0.018804,...,0.037029,0.036804,0.036674,0.036470,0.036404,0.036336,0.036159,0.036074,0.035927,0.035855
1,0.019123,0.019114,0.019107,0.019102,0.019099,0.019100,0.019068,0.019036,0.019009,0.018977,...,0.033504,0.033287,0.033163,0.032966,0.032901,0.032836,0.032665,0.032583,0.032441,0.032370
2,0.018604,0.018595,0.018588,0.018583,0.018580,0.018583,0.018554,0.018526,0.018503,0.018474,...,0.025959,0.025796,0.025703,0.025557,0.025508,0.025456,0.025327,0.025262,0.025154,0.025098
3,0.017203,0.017197,0.017192,0.017189,0.017186,0.017187,0.017161,0.017136,0.017114,0.017088,...,0.031878,0.031700,0.031598,0.031437,0.031385,0.031327,0.031183,0.031111,0.030991,0.030929
4,0.016754,0.016748,0.016742,0.016738,0.016734,0.016733,0.016706,0.016678,0.016655,0.016628,...,0.021612,0.021498,0.021433,0.021331,0.021297,0.021258,0.021164,0.021116,0.021036,0.020994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20195,0.102893,0.102705,0.102512,0.102324,0.102095,0.101906,0.101702,0.101494,0.101290,0.101085,...,0.194174,0.194163,0.194160,0.194149,0.194150,0.193940,0.193766,0.193556,0.193343,0.193175
20196,0.063157,0.063071,0.062981,0.062895,0.062786,0.062699,0.062595,0.062488,0.062384,0.062278,...,0.128498,0.128474,0.128467,0.128447,0.128453,0.128324,0.128207,0.128080,0.127945,0.127851
20197,0.059528,0.059438,0.059346,0.059256,0.059145,0.059055,0.058946,0.058835,0.058728,0.058618,...,0.113018,0.112985,0.112970,0.112942,0.112939,0.112819,0.112704,0.112583,0.112453,0.112359
20198,0.077145,0.077020,0.076891,0.076766,0.076612,0.076487,0.076344,0.076199,0.076057,0.075913,...,0.149373,0.149353,0.149347,0.149330,0.149333,0.149176,0.149040,0.148884,0.148722,0.148601


In [18]:
# Hybrid data preparation RTM data # 


w = db.loc[:,:'Site'] 


inter = feature_preparation(features, order=1)

# list_clean = [1772,1774,2001,3953,3952,3956,5331]+ [i for i in range(1653,1677)]+[i for i in range(1775,1778)] ######## this is specific to the last version of data
# cleansp = inter.drop(list_clean, axis = 0).copy() ######## this is specific to the last version of data

cleansp = inter.copy()
cleanlb = labels.loc[cleansp.index,:].reset_index(drop=True)
cleanw = w.loc[cleansp.index,:].reset_index(drop=True)

X = pd.concat([cleanlb, cleansp.reset_index(drop=True)], axis=1, join='inner')
# X.to_csv(path_n + 'completeClean.csv')

X = X.drop('Year', axis=1) # extra column might cause training issues

# Save processed rtm data
X.to_csv("processed_rtm_data.csv")


X

,Species,dataset,numSamples,LandCover,Tool,Site,Al content (mg/cm²),Aluminum concentration (mg/g),Anthocyanin content (μg/cm²),Anthocyanin concentration (mg/g),...,2441,2442,2443,2444,2445,2446,2447,2448,2449,2450
0,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.37828,NaN,...,0.053722,0.053304,0.052886,0.052468,0.052050,0.051633,0.051215,0.050797,0.050379,0.049961
1,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.34498,NaN,...,0.049425,0.049019,0.048613,0.048207,0.047802,0.047396,0.046990,0.046584,0.046178,0.045772
2,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.12516,NaN,...,0.038084,0.037750,0.037415,0.037081,0.036747,0.036412,0.036078,0.035744,0.035409,0.035075
3,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.38656,NaN,...,0.045201,0.044852,0.044503,0.044153,0.043804,0.043455,0.043106,0.042757,0.042408,0.042058
4,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.62973,NaN,...,0.030430,0.030169,0.029908,0.029647,0.029386,0.029125,0.028865,0.028604,0.028343,0.028082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20195,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.24821,NaN,...,0.201617,0.201331,0.201045,0.200759,0.200473,0.200187,0.199901,0.199615,0.199329,0.199043
20196,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.15780,NaN,...,0.135534,0.135281,0.135027,0.134773,0.134519,0.134266,0.134012,0.133758,0.133504,0.133251
20197,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.08414,NaN,...,0.119755,0.119513,0.119271,0.119029,0.118787,0.118546,0.118304,0.118062,0.117820,0.117578
20198,NaN,43.0,16482.0,NaN,NaN,empty,NaN,NaN,1.17632,NaN,...,0.156390,0.156128,0.155867,0.155606,0.155345,0.155084,0.154823,0.154562,0.154301,0.154039
